# Hyperparameter Tuning by Grid-Search

In the previous notebook, we saw that hyperparameters can affect the generalization performance of a model. In this notebook, we will show how to optimize hyperparameters using  grid-search approach

## Our Predictive Model

Let's reload the dataset as we did previously:

In [1]:
import pandas as pd

from sklearn import set_config
set_config(display="diagram")

In [2]:
df = pd.read_csv('data/adult-census.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
target = df['class']
data = df.drop(columns=['class', 'education-num', 'fnlwgt'])

Once the dataset is loaded, we split it into a training and testing sets.

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
data_train, data_test, target_train, target_test = train_test_split(data, target,
                                                                   random_state=42)

We will define a pipeline as seen in the first module. It will handle both numerical and categorical features.  
The first step is to select all the categorical columns.

In [6]:
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include='object')
categorical_columns = categorical_columns_selector(data)

Here we will use a tree-based model as a classifier (i.e `HisGradientBoostingClassifier`). That means:

* Numerical variables don't need scaling;
* Categorical variables can be dealt with an `OrdinalEncoder` even if the coding order is not meaningful;
* For tree-based models, the `OrdinaleEncoder` avoids having high-dimensional representations.

We now build our `OrdinalEncoder` by passing it the known categories.

In [7]:
from sklearn.preprocessing import OrdinalEncoder

categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value", 
                                          unknown_value=-1)

We then use a `ColumnTransformer` to select the categorical columns and apply the `OrdinalEncoder` to them.

In [8]:
from sklearn.compose import ColumnTransformer

In [9]:
preprocessor = ColumnTransformer([
    ('cat_preprocessor', categorical_preprocessor, categorical_columns)],
    remainder='passthrough', sparse_threshold=0)

Finally, we use a tree-based classifier (i.e. histogram gradient-boosting) to predict whether or not a person earns more than 50k\\$ a year.

In [10]:
# for the moment this line is reuqired to import HistGradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

In [11]:
model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', HistGradientBoostingClassifier(random_state=42, max_leaf_nodes=4))
])

In [12]:
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                                   transformers=[('cat_preprocessor',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier',
                 HistGradientBoostingClassifier(max_leaf_nodes=4,
                                                random_state=42))])

## Tuning Using a Grid-Search

In the previous exercise we use one `for` loop for each hyperparameter to find the best combination over a fixed grid of values.  
`GridSearchCV` is a scikit-learn class that implements a very similar logic with less repetitive code.  

Let's see how to use the `GridSearchCV` estimator for doing such search. Since grid-search will be costly, we will only explore the combination learning rate and the maximum number of nodes.

In [16]:
# Let's display our model parameters first:
model.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'preprocessor', 'classifier', 'preprocessor__n_jobs', 'preprocessor__remainder', 'preprocessor__sparse_threshold', 'preprocessor__transformer_weights', 'preprocessor__transformers', 'preprocessor__verbose', 'preprocessor__cat_preprocessor', 'preprocessor__cat_preprocessor__categories', 'preprocessor__cat_preprocessor__dtype', 'preprocessor__cat_preprocessor__handle_unknown', 'preprocessor__cat_preprocessor__unknown_value', 'classifier__categorical_features', 'classifier__early_stopping', 'classifier__l2_regularization', 'classifier__learning_rate', 'classifier__loss', 'classifier__max_bins', 'classifier__max_depth', 'classifier__max_iter', 'classifier__max_leaf_nodes', 'classifier__min_samples_leaf', 'classifier__monotonic_cst', 'classifier__n_iter_no_change', 'classifier__random_state', 'classifier__scoring', 'classifier__tol', 'classifier__validation_fraction', 'classifier__verbose', 'classifier__warm_start'])

In [17]:
%%time
from sklearn.model_selection import GridSearchCV

param_grid = {
    'classifier__learning_rate': (.01, .1, 1, 10),
    'classifier__max_leaf_nodes': (3, 10, 30)}

model_grid_search = GridSearchCV(model, param_grid=param_grid,
                                 n_jobs=2, cv=2)
model_grid_search.fit(data_train, target_train)

CPU times: user 4.44 s, sys: 446 ms, total: 4.89 s
Wall time: 13.2 s


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          sparse_threshold=0,
                                                          transformers=[('cat_preprocessor',
                                                                         OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                        unknown_value=-1),
                                                                         ['workclass',
                                                                          'education',
                                                                          'marital-status',
                                                                          'occupation',
                                                                          'relationship',
                                                                          'race',
                                                                          'sex',
                                                                          'native-country'])])),
                                       ('classifier',
                                        HistGradientBoostingClassifier(max_leaf_nodes=4,
                                                                       random_state=42))]),
             n_jobs=2,
             param_grid={'classifier__learning_rate': (0.01, 0.1, 1, 10),
                         'classifier__max_leaf_nodes': (3, 10, 30)})

Finally, we will check the accuracy of our model using the test set.

In [18]:
accuracy = model_grid_search.score(data_test, target_test)
print(
    f"The test accuracy score of the grid-searched pipeline is: "
    f"{accuracy:.2f}"
)

The test accuracy score of the grid-searched pipeline is: 0.88


<div class="alert alert-block alert-warning">
<b>Warning:</b> Be aware that evaluation should normally be performed in a cross-validation framework by providing <em>model_grid_search</em> as a model to the <em>cross_validate</em> function.
</div>

The `GridSearchCV` estimator takes a `param_grid` parameter which defines all hyperparameters and their associated values. The grid-search will be in charge of creating all possible combinations and test them.

Once the grid-search is fitted, it can be used as any other predictor by calling `predict` and `predict_proba`. Internally, it will use the model with the best parameters found during `fit`.

Get predictions for the 5 first samples using the estimator with the best parameters.

In [19]:
model_grid_search.predict(data_test[:5])

array([' <=50K', ' <=50K', ' >50K', ' <=50K', ' >50K'], dtype=object)

In [21]:
print(target_test[:5])

7762      <=50K
23881     <=50K
30507      >50K
28911     <=50K
19484     <=50K
Name: class, dtype: object


You can know about these parameters by looking at the `best_params_` attribute.

In [22]:
print(f"The best set of parameters is: "
      f"{model_grid_search.best_params_}")

The best set of parameters is: {'classifier__learning_rate': 0.1, 'classifier__max_leaf_nodes': 30}
